In [49]:
import piezo
import pandas as pd
from catomatic.CatalogueBuilder import BuildCatalogue

This notebook demonstrates how to use catomatic, and its various functionalities - a number of non-trivial decisions need to be made depending on the data and context, which are covered here.

# Import mutations and samples tables

In [50]:
mutations = pd.read_csv('../data/mutations.csv', index_col=0)
mutations

,UNIQUEID,MUTATION,FRS
0,1,gene@A1S,0.73
1,2,gene@A1S,0.25
2,4,gene@A2S,0.40
3,7,gene@A3S,0.98
4,8,gene@A3S,0.90
5,9,gene@A3S,0.86


In [51]:
samples = pd.read_csv('../data/samples.csv', index_col=0)
samples

,UNIQUEID,PHENOTYPE
0,1,R
1,2,S
2,3,R
3,4,S
4,5,R
5,6,S
6,7,S
7,8,S
8,9,R
9,10,R


# Build the catalogue

In [52]:
catalogue = BuildCatalogue(samples=samples, mutations=mutations)

Can return as a dictionary, where each key is a mutations:

In [53]:
catalogue.return_catalogue()

{'gene@A2S': {'pred': 'S',
  'evid': ({'proportion': 0.0,
    'confidence': (0.0, 0.7934506856227626),
    'contingency': [[0, 1], [3, 1]]},)},
 'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'contingency': [[1, 2], [3, 2]]},)}}

Or can return as a piezo-structure dataframe:

In [54]:
#note, wildcards can be supplied as a dictionary or path to json. for_piezo will add placeholder rows to avoid an issue with piezo (requires R+U+S)
catalogue.build_piezo(genbank_ref='demo_gene', catalogue_name='demo', version='0.1.1', drug='eg', wildcards='../data/demo_wildcards.json', for_piezo=False)

,GENBANK_REFERENCE,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,PREDICTION_VALUES,DRUG,MUTATION,PREDICTION,SOURCE,EVIDENCE,OTHER
0,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@A2S,S,{},"[{""proportion"": 0.0, ""confidence"": [0.0, 0.793...",{}
1,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@A1S,U,{},"[{""proportion"": 0.5, ""confidence"": [0.09453120...",{}
2,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@A3S,U,{},"[{""proportion"": 0.3333333333333333, ""confidenc...",{}
3,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@*=,S,{},"{""default_rule"": ""True""}",{}
4,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@-*_indel,U,{},"{""default_rule"": ""True""}",{}
5,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@*_indel,U,{},"{""default_rule"": ""True""}",{}
6,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@-*?,U,{},"{""default_rule"": ""True""}",{}
7,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@*?,U,{},"{""default_rule"": ""True""}",{}
8,demo_gene,demo,0.1.1,GARC1,RUS,eg,gene@del_0.0,U,{},"{""default_rule"": ""True""}",{}


Or can call 'to_piezo' to directly save as a csv:

In [55]:
catalogue.to_piezo(genbank_ref='demo_gene', catalogue_name='demo', version='0.1.1', drug='eg', wildcards='../data/demo_wildcards.json', outfile='./temp/demo_catalogue.csv')

Piezo can then be used to make predictions:

In [56]:
catalogue = piezo.ResistanceCatalogue("./temp/demo_catalogue.csv")
print ('prediction for mutation gene@A1S:', catalogue.predict('gene@A1S'))
print ('prediction for mutation gene@A2S:', catalogue.predict('gene@A2S'))


prediction for mutation gene@A1S: {'eg': 'U'}
prediction for mutation gene@A2S: {'eg': 'S'}


## Statistical Tests

The default parameter for test type is `None`, which instantiates a protocol whereby susceptible mutations are defined as those with homogenously susceptible phenotypes at the level at which it is being classified - this facilitates the unlocking mechanism, however beyond that, no phenotyping is done.

To classify resistant variants, one can run a traditional binomial test on the proportion of resistance observed at that level against a user-defined background rate, by specifying `Binomial` and a background.

In [57]:
#run a binomial test with a background rate of 20% and a 90% 2-tailed confidence level (p=0.9) - notice R classification.
catalogue = BuildCatalogue(samples=samples, mutations=mutations, test='Binomial', background=0.05, p=0.9)
catalogue.build_piezo(genbank_ref='demo_gene', catalogue_name='demo_binomial', version='0.1.1', drug='eg', wildcards='../data/demo_wildcards.json')

,GENBANK_REFERENCE,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,PREDICTION_VALUES,DRUG,MUTATION,PREDICTION,SOURCE,EVIDENCE,OTHER
0,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@A2S,S,{},"[{""proportion"": 0.0, ""confidence"": [0.0, 0.730...",{}
1,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@A1S,R,{},"[{""proportion"": 0.5, ""confidence"": [0.12086631...",{}
2,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@A3S,U,{},"[{""proportion"": 0.3333333333333333, ""confidenc...",{}
3,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*=,S,{},"{""default_rule"": ""True""}",{}
4,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@-*_indel,U,{},"{""default_rule"": ""True""}",{}
5,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*_indel,U,{},"{""default_rule"": ""True""}",{}
6,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@-*?,U,{},"{""default_rule"": ""True""}",{}
7,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*?,U,{},"{""default_rule"": ""True""}",{}
8,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@del_0.0,U,{},"{""default_rule"": ""True""}",{}


One can also run a Fisher's test against a calculated background rate from a contintengcy table by specifying test type `Fisher`. 

Backgrounds are defined as R or S phenotypes in the absence of that mutation or any mutations not yet classified as susceptible (ie, background = 'WT' phenotypes + classified S phenotypes). Essentially, the background proportion of resistance is that which we cannot logically explain and probably arises from error or missing information.

In [58]:
#run a Fisher test at 10% 2-tailed confidence level (p=0.1) - notice the S classification in the second row (a crazy p-value, I realise - for demonstration). 
catalogue = BuildCatalogue(samples=samples, mutations=mutations, test='Fisher', p=0.1)
catalogue.build_piezo(genbank_ref='demo_gene', catalogue_name='demo_fisher', version='0.1.1', drug='eg', wildcards='../data/demo_wildcards.json')

,GENBANK_REFERENCE,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,PREDICTION_VALUES,DRUG,MUTATION,PREDICTION,SOURCE,EVIDENCE,OTHER
0,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@A2S,S,{},"[{""proportion"": 0.0, ""confidence"": [0.0, 0.015...",{}
1,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@A3S,S,{},"[{""proportion"": 0.3333333333333333, ""confidenc...",{}
2,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@A1S,U,{},"[{""proportion"": 0.5, ""confidence"": [0.45574636...",{}
3,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@*=,S,{},"{""default_rule"": ""True""}",{}
4,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@-*_indel,U,{},"{""default_rule"": ""True""}",{}
5,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@*_indel,U,{},"{""default_rule"": ""True""}",{}
6,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@-*?,U,{},"{""default_rule"": ""True""}",{}
7,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@*?,U,{},"{""default_rule"": ""True""}",{}
8,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,gene@del_0.0,U,{},"{""default_rule"": ""True""}",{}
9,demo_gene,demo_fisher,0.1.1,GARC1,RUS,eg,placeholder@R1R,R,{},{},{}


 This is particularly powerful if a meaningful background rate can actually be calculated - ie one that represents sample phenotyping error + labelling error + unexplainable resistance mechanisms etc.

 However, in practise often they don't actually represent anything, and an assumed error rate is needed.

 Furthemore, clean and accurate datasets for drugs with well-defined resistance mechanisms will have very low background rates, which when using a Fisher's test can lead to Resistant classifications at very low proportions (like 5%). This can be unhelpful, and there are 2 ways to overcome this. Firstly, one can raise the p-value to increase statistical stringency, or one can use a binomial test with a higher, defined background.

## Seeding

A number of genes contain phylogeneitc, neutral mutations which have persisted for a considerable length of time and are largely assumed to not influence the resistant phenotype (e.g: https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00726-5).

Due to the high prevelance of these mutations, and the fact the algorithm uses variants that occur in isolation to search for resistance-drivers, they ultimately just add noise.

Therefore, catomatic allows for seeding, whereby pre-defined neutral variants can be added prior to running the algorithm:

In [59]:
#seed the method with gene@A5D and gene@A6T
catalogue = BuildCatalogue(samples=samples, mutations=mutations, seed=['gene@A5D', 'gene@A6T'])
catalogue.return_catalogue()

{'gene@A5D': {'pred': 'S', 'evid': 'seeded'},
 'gene@A6T': {'pred': 'S', 'evid': 'seeded'},
 'gene@A2S': {'pred': 'S',
  'evid': ({'proportion': 0.0,
    'confidence': (0.0, 0.7934506856227626),
    'contingency': [[0, 1], [3, 1]]},)},
 'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'contingency': [[1, 2], [3, 2]]},)}}

## Fraction Read Support

Post-sequencing bioinformatics pipelines often contain steps to reduce sequencing error. One such step implements a filter for the fraction of genetic reads that support a particular variant call at that position. 

However, certain genes can display heterogenous alleles which, should they contain a resistant mutation, can outcompete majority populations under the selection pressure of drug administration, regardless of how few observed reads support that allele. 

Catomatic therefore supports FRS thresholding, whereby only variants with read support above a user-supplied threshold are considered:

In [60]:
#Build a catalogue for variants which contained over 90% read support - ie, genetic homogeneity - notice only gene@A3S is catalogued
catalogue = BuildCatalogue(samples=samples, mutations=mutations, test='Binomial', background=0.05, p=0.9, FRS=0.9)
catalogue.build_piezo(genbank_ref='demo_gene', catalogue_name='demo_binomial', version='0.1.1', drug='eg', wildcards='../data/demo_wildcards.json')

,GENBANK_REFERENCE,CATALOGUE_NAME,CATALOGUE_VERSION,CATALOGUE_GRAMMAR,PREDICTION_VALUES,DRUG,MUTATION,PREDICTION,SOURCE,EVIDENCE,OTHER
0,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@A3S,S,{},"[{""proportion"": 0.0, ""confidence"": [0.0, 0.574...",{}
1,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*=,S,{},"{""default_rule"": ""True""}",{}
2,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@-*_indel,U,{},"{""default_rule"": ""True""}",{}
3,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*_indel,U,{},"{""default_rule"": ""True""}",{}
4,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@-*?,U,{},"{""default_rule"": ""True""}",{}
5,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@*?,U,{},"{""default_rule"": ""True""}",{}
6,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,gene@del_0.0,U,{},"{""default_rule"": ""True""}",{}
7,demo_gene,demo_binomial,0.1.1,GARC1,RUS,eg,placeholder@R1R,R,{},{},{}


## Strict S variant Unlocking

If one were to phenotype based only on statistically significant resistance proportions, one should specify `strict_unlock=True`. This setting constrains the susceptible variant search step to require a calculated p-value less than the specified threshold.

However, in practice, many variants from clinical studies are observed infrequently. If `strict_unlock=True` is set, all singletons and many low-frequency variants would be discarded due to frequentist statistics requiring a minimum number of observations to achieve sufficient statistical power. This requirement ensures that the statistical tests are robust and the results are reliable, but it also means that variants with low observation numbers are often excluded from analysis.

One may, therefore, decide to use a looser definition of 'difference' and accept calling variants susceptible if there is no observed evidence of resistance and no statistical evidence of resistance. Essentially, setting `strict_unlock=False` allows homogenous susceptible (S) variants to be called susceptible (S), regardless of the total number of variant observations.

In [61]:
#notice gene@A2S is called susceptible....
catalogue = BuildCatalogue(samples=samples, mutations=mutations, test='Fisher', strict_unlock=False)
catalogue.return_catalogue()

{'gene@A2S': {'pred': 'S',
  'evid': ({'proportion': 0.0,
    'confidence': (0.0, 0.7934506856227626),
    'p_value': 0.4000000000000001,
    'contingency': [[0, 1], [3, 1]]},)},
 'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'p_value': 1.0,
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'p_value': 1.0,
    'contingency': [[1, 2], [3, 2]]},)}}

In [62]:
#... but now does not pass the statistical requirement for phenotyping
catalogue = BuildCatalogue(samples=samples, mutations=mutations, test='Fisher', strict_unlock=True)
catalogue.return_catalogue()

{'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'p_value': 1.0,
    'contingency': [[1, 1], [3, 1]]},)},
 'gene@A2S': {'pred': 'U',
  'evid': ({'proportion': 0.0,
    'confidence': (0.0, 0.7934506856227626),
    'p_value': 0.4000000000000001,
    'contingency': [[0, 1], [3, 1]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'p_value': 0.48571428571428565,
    'contingency': [[1, 2], [3, 1]]},)}}

## post-hoc updates

Should one wish to overried a variant call in the catalogue, it can be done by simply supplying a key-value pair of the mutation:phenotype, which replace the current row in the catalogue.

This level of blatent expert-intervention is old-fashioned and strongly discouraged, and could probably be avoided via tuning the various parameters above, and certainly warrants an investigation as to why that variant is being phenotyped incorreclty (which can be partly deduced by the evidence logged in that row).



In [63]:
catalogue = BuildCatalogue(samples=samples, mutations=mutations)
catalogue.return_catalogue()

{'gene@A2S': {'pred': 'S',
  'evid': ({'proportion': 0.0,
    'confidence': (0.0, 0.7934506856227626),
    'contingency': [[0, 1], [3, 1]]},)},
 'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'contingency': [[1, 2], [3, 2]]},)}}

In [64]:
catalogue.update({'gene@A2S':'R'})
catalogue.return_catalogue()

{'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'contingency': [[1, 2], [3, 2]]},)},
 'gene@A2S': {'pred': 'R', 'evid': {}}}

One can also add wildcard rules to the catalogue - for example, if there is evidence that all frameshift mutations are resistant (as is sometimes the case in non-essential genes).

Simply appending it would result in a piezo-facilitated hierarchy during the piezo predictions, whereby more specific mutation definitions have priority over wildcard rules. Therefore, one could add the rule and any specific mutations that do not agree will be called seperately.

In [65]:
#notice the rule is simply appended, and will have lower priority than the more specific rows above
catalogue.update({'gene@*?':'R'})
catalogue.return_catalogue()

{'gene@A1S': {'pred': 'U',
  'evid': ({'proportion': 0.5,
    'confidence': (0.09453120573423071, 0.9054687942657693),
    'contingency': [[1, 1], [3, 2]]},)},
 'gene@A3S': {'pred': 'U',
  'evid': ({'proportion': 0.3333333333333333,
    'confidence': (0.061491944720396215, 0.7923403991979523),
    'contingency': [[1, 2], [3, 2]]},)},
 'gene@A2S': {'pred': 'R', 'evid': {}},
 'gene@*?': {'pred': 'R', 'evid': {}}}

Alternatively, one can specify `replace=True`, which will find all variants that fall under that rule and remove them before appending the rule. Note, because the search mechanism uses piezo, a set of wildcards have to be supplied (even though they won't be added to the catalogue.)

In [66]:
#add a wildcard rule with replacement - all example variants fall under this rule, so are replaced
catalogue = BuildCatalogue(samples=samples, mutations=mutations)
catalogue.update({'gene@*?':'R'}, replace=True, wildcards='../data/demo_wildcards.json')
catalogue.return_catalogue()

{'gene@*?': {'pred': 'R', 'evid': {}}}